## Model Choice Overview

### EDA-informed top features
1. `number_inpatient`
2. `number_emergency`
3. `number_outpatient`
4. `number_diagnoses`
5. `admission_source_id`

### Context
- Class imbalance exists (`NO` dominates), therefore macro metrics and recall monitoring are mandatory.
- Interpretability matters for hospital stakeholders, so transparent baselines remain in scope.


| Dataset Characteristic | Primary Algorithm | Secondary Algorithm | Rationale |
|---|---|---|---|
| Medium tabular dataset | Random Forest | Logistic Regression | Non-linear signal capture + interpretable baseline |
| High-dimensional encoded features | Logistic Regression | Random Forest | Linear model handles sparse one-hot well; RF checks for interactions |
| Class imbalance | SMOTE + RF/XGB | Class-weighted Logistic Regression | Improve minority recall while keeping a transparent reference |
| Complex feature interactions | XGBoost (if available) | Random Forest | Boosting often wins on tabular data; RF is a solid fallback |
| Stakeholder explainability need | Logistic Regression | Calibrated tree-based model | Easier policy discussion and threshold setting |


Import Models

In [9]:
import sys
import importlib
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from scipy.stats import randint, uniform
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

PROJECT_ROOT = Path.cwd().resolve().parent
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import notebook_checks as notebook_checks_mod
import styling.styling as styling_mod
import model_pipeline as model_pipeline_mod

importlib.reload(notebook_checks_mod)
importlib.reload(styling_mod)
importlib.reload(model_pipeline_mod)

from config import get_config

build_model_candidates = model_pipeline_mod.build_model_candidates
compute_binary_pr_curve = model_pipeline_mod.compute_binary_pr_curve
compute_error_analysis = model_pipeline_mod.compute_error_analysis
establish_dummy_baselines = model_pipeline_mod.establish_dummy_baselines
evaluate_models_cv = model_pipeline_mod.evaluate_models_cv
fit_and_evaluate_test = model_pipeline_mod.fit_and_evaluate_test
load_model_data = model_pipeline_mod.load_model_data
run_model_pipeline = model_pipeline_mod.run_model_pipeline
feature_error_patterns = model_pipeline_mod.feature_error_patterns
confidence_strategy = model_pipeline_mod.confidence_strategy
check_calibration = model_pipeline_mod.check_calibration

assert_feature_alignment = notebook_checks_mod.assert_feature_alignment
assert_no_missing_values = notebook_checks_mod.assert_no_missing_values
assert_valid_target_labels = notebook_checks_mod.assert_valid_target_labels
build_reproducibility_footer = notebook_checks_mod.build_reproducibility_footer

apply_notebook_style = styling_mod.apply_notebook_style
build_eda_output_paths = styling_mod.build_eda_output_paths
build_artifact_path = styling_mod.build_artifact_path
save_table_snapshot = styling_mod.save_table_snapshot


ModuleNotFoundError: No module named 'sklearn.utils._test_common.instance_generator'

Import Data

In [ ]:
cfg = get_config(PROJECT_ROOT)
apply_notebook_style()
output_path, table_output_path = build_eda_output_paths(PROJECT_ROOT)

NOTEBOOK_ID = '03'
def fig_path(section_id: str, slug: str):
    return build_artifact_path(output_path, NOTEBOOK_ID, section_id, slug, 'png')

def table_path(section_id: str, slug: str, extension: str = 'png'):
    return build_artifact_path(table_output_path, NOTEBOOK_ID, section_id, slug, extension)

data = load_model_data(cfg, include_binary=True)

X_train = data.X_train
X_test = data.X_test
y_train = data.y_train
y_test = data.y_test
y_train_binary = data.y_train_binary
y_test_binary = data.y_test_binary

assert_feature_alignment(X_train, X_test)
assert_no_missing_values(X_train, 'X_train_encoded')
assert_no_missing_values(X_test, 'X_test_encoded')
assert_valid_target_labels(y_train, 'y_train_encoded', allowed_labels=(0, 1, 2))
assert_valid_target_labels(y_test, 'y_test_encoded', allowed_labels=(0, 1, 2))

print(f"X_train: {X_train.shape}")
print(f"X_test : {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test : {y_test.shape}")
if y_test_binary is not None:
    print(f"y_test_binary : {y_test_binary.shape}")


## Why These Models?

- **Logistic Regression**: strongest interpretability baseline, useful for stakeholder trust.
- **Random Forest**: robust non-linear tabular benchmark with low preprocessing sensitivity.
- **XGBoost (if available)**: performance-oriented candidate for complex interaction capture.

All are evaluated with consistent CV and test metrics under class-imbalance handling.


## Baseline Model

### Baseline Strategy

Level 1 (dummy baselines): sanity check that learned models beat naive strategies.

Level 2 (candidate models): compare interpretable vs high-capacity models using the same CV setup.


In [ ]:
baseline_results = establish_dummy_baselines(
    X_train,
    y_train,
    cv_folds=5,
    random_state=cfg.random_state,
)

display(baseline_results.sort_values('cv_f1_macro_mean', ascending=False))


### Candidate Model Training and Evaluation


In [ ]:
pipeline_results = run_model_pipeline(
    config=cfg,
    cv_folds=5,
    include_baselines=True,
    save_artifacts=True,
)

cv_results = pipeline_results['cv_results']
test_results = pipeline_results['test_results']
fitted_models = pipeline_results['fitted_models']
best_model_name = pipeline_results['best_model_name']
best_model = pipeline_results['best_model']
model_reports = pipeline_results['model_reports']
saved_paths = pipeline_results['saved_paths']

print(f"Best model by CV priority (<30 recall, then precision): {best_model_name}")
print(f"Saved best model path: {saved_paths.get('best_model', cfg.model_path)}")



In [ ]:
y_pred_best = best_model.predict(X_test)

print('Classification report (best untuned model):')
print(classification_report(y_test, y_pred_best, digits=4))

cm = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix - {best_model_name} (Untuned)')
plt.xlabel('Predicted class')
plt.ylabel('Actual class')
plt.tight_layout()
plt.savefig(fig_path('04', 'confusion_matrix_untuned'), dpi=220)
plt.show()


### Different Models

We train and compare multiple candidates (linear + non-linear) using the same preprocessing, CV setup, and test split:

- **Dummy baselines**: sanity-check performance floor.
- **Logistic Regression** (linear): interpretability and fast iteration.
- **Random Forest** (non-linear): interaction capture with minimal feature scaling.
- **XGBoost** (non-linear, optional): high-capacity tabular model when the dependency is available.

Selection priority follows the problem objective: maximize recall on the `<30` class first, then macro precision and macro F1 to control false positives across classes.


In [ ]:
comparison_df = cv_results.merge(test_results, on='model', how='left')
comparison_df = comparison_df.sort_values('cv_f1_macro_mean', ascending=False)

display(comparison_df)

save_table_snapshot(
    comparison_df,
    table_path('04', 'model_comparison_summary'),
    title='Model Choice - CV/Test Comparison',
    index=False,
)
comparison_df.to_csv(table_path('04', 'model_comparison_summary', 'csv'), index=False)


In [ ]:
if y_test_binary is None:
    print('Binary target files not found. Skipping PR analysis.')
else:
    plt.figure(figsize=(8, 6))
    for model_name, model in fitted_models.items():
        pr_df, ap = compute_binary_pr_curve(model, X_test, y_test_binary)
        plt.plot(pr_df['recall'], pr_df['precision'], label=f"{model_name} (AP={ap:.3f})")

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curves (Readmitted vs No Readmission)')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(fig_path('04', 'precision_recall_curves_all_models'), dpi=220)
    plt.show()


### ROC Curves (Binary proxy target, if available)


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

if y_test_binary is None:
    print('Binary target files not found. Skipping ROC analysis.')
else:
    unique_y = np.unique(y_test_binary)
    if len(unique_y) < 2:
        print(f'Binary target has only one class ({unique_y}). Skipping ROC analysis.')
    else:
        plt.figure(figsize=(8, 6))
        for model_name, model in fitted_models.items():
            y_score = None
            if hasattr(model, "predict_proba"):
                y_score = model.predict_proba(X_test)[:, 1]
            elif hasattr(model, "decision_function"):
                y_score = model.decision_function(X_test)
            if y_score is None:
                continue
            fpr, tpr, _ = roc_curve(y_test_binary, y_score)
            auc = roc_auc_score(y_test_binary, y_score)
            plt.plot(fpr, tpr, label=f"{model_name} (AUC={auc:.3f})")

        plt.plot([0, 1], [0, 1], "k--", alpha=0.4)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves (Readmitted vs No Readmission)')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(fig_path('04', 'roc_curves_all_models'), dpi=220)
        plt.show()


In [ ]:
recall_rows = []
for model_name, report in model_reports.items():
    recall_rows.append(
        {
            'model': model_name,
            'recall_class_0_NO': report.get('0', {}).get('recall', np.nan),
            'recall_class_1_GT30': report.get('1', {}).get('recall', np.nan),
            'recall_class_2_LT30': report.get('2', {}).get('recall', np.nan),
            'recall_macro': report.get('macro avg', {}).get('recall', np.nan),
        }
    )

recall_breakdown = pd.DataFrame(recall_rows).sort_values('recall_macro', ascending=False)
display(recall_breakdown)

save_table_snapshot(
    recall_breakdown,
    table_path('04', 'recall_breakdown_by_class'),
    title='Model Choice - Recall Breakdown by Class',
    index=False,
)
recall_breakdown.to_csv(table_path('04', 'recall_breakdown_by_class', 'csv'), index=False)


## Hyperparameter Tuning


In [ ]:
from sklearn.metrics import make_scorer, recall_score

models_for_tuning = build_model_candidates(cfg.random_state)
base_tune_model = models_for_tuning[best_model_name]

if best_model_name == 'LogisticRegression':
    param_distributions = {
        'model__C': uniform(0.01, 10),
        'model__penalty': ['l2'],
    }
elif best_model_name == 'RandomForest':
    param_distributions = {
        'model__n_estimators': randint(150, 500),
        'model__max_depth': randint(4, 30),
        'model__min_samples_split': randint(2, 20),
        'model__min_samples_leaf': randint(1, 10),
    }
else:
    # XGBoost branch
    param_distributions = {
        'model__n_estimators': randint(150, 500),
        'model__max_depth': randint(3, 10),
        'model__learning_rate': uniform(0.01, 0.20),
        'model__subsample': uniform(0.6, 0.4),
        'model__colsample_bytree': uniform(0.6, 0.4),
    }

print('RandomizedSearchCV search space (param_distributions):')
display(pd.Series({k: str(v) for k, v in param_distributions.items()}, name='distribution').to_frame())

priority_scoring = {
    'recall_lt30': make_scorer(recall_score, labels=[2], average='macro', zero_division=0),
    'precision_macro': 'precision_macro',
    'f1_macro': 'f1_macro',
}

def refit_lt30_recall_then_precision(cv_results):
    ranking = (
        pd.DataFrame(cv_results)
        .assign(_idx=lambda d: d.index)
        .sort_values(
            ['mean_test_recall_lt30', 'mean_test_precision_macro', 'mean_test_f1_macro'],
            ascending=[False, False, False],
        )
    )
    return int(ranking.iloc[0]['_idx'])

random_search = RandomizedSearchCV(
    estimator=base_tune_model,
    param_distributions=param_distributions,
    n_iter=15,
    scoring=priority_scoring,
    refit=refit_lt30_recall_then_precision,
    cv=5,
    random_state=cfg.random_state,
    n_jobs=1,
    verbose=1,
)
random_search.fit(X_train, y_train)

tuned_model = random_search.best_estimator_
y_pred_tuned = tuned_model.predict(X_test)

best_idx = random_search.best_index_
best_cv_recall_lt30 = float(random_search.cv_results_['mean_test_recall_lt30'][best_idx])
best_cv_precision_macro = float(random_search.cv_results_['mean_test_precision_macro'][best_idx])
best_cv_f1_macro = float(random_search.cv_results_['mean_test_f1_macro'][best_idx])

print('Best params:')
print(random_search.best_params_)
print(f"Best CV recall_lt30: {best_cv_recall_lt30:.4f}")
print(f"Best CV precision_macro: {best_cv_precision_macro:.4f}")
print(f"Best CV f1_macro: {best_cv_f1_macro:.4f}")



Tuning objective: maximize `f1_macro` while preserving minority-class recall.


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib

tuned_test_f1 = f1_score(y_test, y_pred_tuned, average='macro')
tuned_test_precision_macro = precision_score(y_test, y_pred_tuned, average='macro', zero_division=0)
tuned_test_recall_lt30 = recall_score(y_test, y_pred_tuned, labels=[2], average='macro', zero_division=0)

summary_row = pd.DataFrame(
    [
        {
            'selected_base_model': best_model_name,
            'best_cv_recall_lt30_tuned': best_cv_recall_lt30,
            'best_cv_precision_macro_tuned': best_cv_precision_macro,
            'best_cv_f1_macro_tuned': best_cv_f1_macro,
            'test_recall_lt30_tuned': tuned_test_recall_lt30,
            'test_precision_macro_tuned': tuned_test_precision_macro,
            'test_f1_macro_tuned': tuned_test_f1,
        }
    ]
)
display(summary_row)

tuned_model_path = cfg.final_dir / f"{best_model_name.lower()}_tuned.joblib"
joblib.dump(tuned_model, tuned_model_path)
print(f"Tuned model saved to: {tuned_model_path}")



In [ ]:
# Feature importance / impact inspection
if hasattr(tuned_model.named_steps['model'], 'feature_importances_'):
    raw_importance = tuned_model.named_steps['model'].feature_importances_
    importance_df = pd.DataFrame({'feature': X_train.columns, 'importance': raw_importance})
    importance_df = importance_df.sort_values('importance', ascending=False)
else:
    coef = tuned_model.named_steps['model'].coef_
    importance = np.abs(coef).mean(axis=0)
    importance_df = pd.DataFrame({'feature': X_train.columns, 'importance': importance})
    importance_df = importance_df.sort_values('importance', ascending=False)

importance_top = importance_df.head(25)
display(importance_top)

save_table_snapshot(
    importance_top,
    table_path('06', 'tuned_model_feature_importance_top25'),
    title='Model Choice - Tuned Model Feature Importance (Top 25)',
    index=False,
)
importance_top.to_csv(table_path('06', 'tuned_model_feature_importance_top25', 'csv'), index=False)


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate

cv_compare_rows = []
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=cfg.random_state)
for label, model in {
    'best_untuned': best_model,
    'best_tuned': tuned_model,
}.items():
    scores = cross_validate(
        model,
        X_train,
        y_train,
        cv=cv_strategy,
        scoring={'f1_macro': 'f1_macro', 'recall_macro': 'recall_macro', 'accuracy': 'accuracy'},
        n_jobs=1,
    )
    cv_compare_rows.append(
        {
            'model_variant': label,
            'cv_f1_macro_mean': scores['test_f1_macro'].mean(),
            'cv_recall_macro_mean': scores['test_recall_macro'].mean(),
            'cv_accuracy_mean': scores['test_accuracy'].mean(),
        }
    )

cv_compare_df = pd.DataFrame(cv_compare_rows)
display(cv_compare_df)

save_table_snapshot(
    cv_compare_df,
    table_path('06', 'tuned_vs_untuned_cv_metrics'),
    title='Model Choice - Tuned vs Untuned CV Metrics',
    index=False,
)
cv_compare_df.to_csv(table_path('06', 'tuned_vs_untuned_cv_metrics', 'csv'), index=False)


## Visualize performance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.barplot(data=comparison_df, x='model', y='cv_f1_macro_mean', ax=axes[0], palette='Blues')
axes[0].set_title('Cross-Validation F1 Macro by Model')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('CV F1 Macro')
axes[0].tick_params(axis='x', rotation=20)

sns.barplot(data=comparison_df, x='model', y='test_f1_macro', ax=axes[1], palette='Greens')
axes[1].set_title('Test F1 Macro by Model')
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Test F1 Macro')
axes[1].tick_params(axis='x', rotation=20)

plt.tight_layout()
plt.savefig(fig_path('07', 'model_f1_comparison_bars'), dpi=220)
plt.show()


In [ ]:
y_pred_tuned = tuned_model.predict(X_test)
cm_tuned = confusion_matrix(y_test, y_pred_tuned)

plt.figure(figsize=(7, 5))
sns.heatmap(cm_tuned, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Tuned Model')
plt.xlabel('Predicted class')
plt.ylabel('Actual class')
plt.tight_layout()
plt.savefig(fig_path('07', 'confusion_matrix_tuned'), dpi=220)
plt.show()

if y_test_binary is not None:
    pr_tuned, ap_tuned = compute_binary_pr_curve(tuned_model, X_test, y_test_binary)
    plt.figure(figsize=(6, 5))
    plt.plot(pr_tuned['recall'], pr_tuned['precision'], label=f'Tuned model AP={ap_tuned:.3f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve - Tuned Model')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(fig_path('07', 'precision_recall_tuned'), dpi=220)
    plt.show()

    # ROC curve for the tuned model (binary proxy target)
    y_score_tuned = None
    if hasattr(tuned_model, 'predict_proba'):
        y_score_tuned = tuned_model.predict_proba(X_test)[:, 1]
    elif hasattr(tuned_model, 'decision_function'):
        y_score_tuned = tuned_model.decision_function(X_test)

    if y_score_tuned is not None:
        unique_y_tuned = np.unique(y_test_binary)
        if len(unique_y_tuned) < 2:
            print(f'Binary target has only one class ({unique_y_tuned}). Skipping tuned ROC.')
        else:
            from sklearn.metrics import roc_auc_score, roc_curve
            fpr_tuned, tpr_tuned, _ = roc_curve(y_test_binary, y_score_tuned)
            auc_tuned = roc_auc_score(y_test_binary, y_score_tuned)
            plt.figure(figsize=(6, 5))
            plt.plot(fpr_tuned, tpr_tuned, label=f'Tuned model AUC={auc_tuned:.3f}')
            plt.plot([0, 1], [0, 1], 'k--', alpha=0.4)
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('ROC Curve - Tuned Model')
            plt.legend()
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(fig_path('07', 'roc_curve_tuned'), dpi=220)
            plt.show()


In [ ]:
untuned_best_row = cv_results.iloc[0]
improvement_recall_lt30 = best_cv_recall_lt30 - float(untuned_best_row['cv_recall_lt30_mean'])
improvement_precision_macro = best_cv_precision_macro - float(untuned_best_row['cv_precision_macro_mean'])

print('Business impact summary')
print(f"Best untuned CV recall_lt30      : {float(untuned_best_row['cv_recall_lt30_mean']):.4f}")
print(f"Best tuned CV recall_lt30        : {best_cv_recall_lt30:.4f}")
print(f"Best untuned CV precision_macro  : {float(untuned_best_row['cv_precision_macro_mean']):.4f}")
print(f"Best tuned CV precision_macro    : {best_cv_precision_macro:.4f}")
print(f"Best tuned CV f1_macro           : {best_cv_f1_macro:.4f}")
print(f"Gain in recall_lt30 from tuning  : {improvement_recall_lt30:.4f}")
print(f"Gain in precision_macro tuning   : {improvement_precision_macro:.4f}")



## Reflect on Model Behavior and Trade-Offs

Key takeaways from CV + test evaluation:

- **Imbalance sensitivity**: Accuracy is dominated by the `NO` class; macro metrics and per-class recall (especially `<30`) are more informative.
- **Linear vs non-linear**: Logistic Regression gives a strong, explainable baseline; tree/boosted models can improve minority recall by capturing interactions, but are harder to explain and may require calibration.
- **Tuning effects**: Randomized search improves the recall/precision trade-off for the selected best model; gains should be validated against overfitting (CV vs test gap).
- **Operational trade-off**: A confidence threshold lets us auto-process high-confidence predictions and route low-confidence cases to review, reducing risk from uncertain predictions.
- **What to carry forward**: Use the tuned version of the selected best model (`best_model_name`, saved as a joblib artifact) as the primary candidate for the next milestone; keep Logistic Regression as a transparency baseline.

Notes for future refinement:

- Data quality (coding noise, label ambiguity) and feature engineering (aggregations, interaction proxies) likely limit minority performance; revisit preprocessing/feature set before heavy tuning.
- Compare class weights vs SMOTE depending on calibration stability and false-positive costs.


### Error Detection and Confidence Strategy

Use error composition and confidence thresholds to decide when automated predictions need clinician review.


In [ ]:
error_summary = compute_error_analysis(
    model=tuned_model,
    X_test=X_test,
    y_test=y_test,
    confidence_threshold=0.80,
)

display(pd.Series(error_summary, name='value').to_frame())


In [ ]:
y_pred_tuned = tuned_model.predict(X_test)
errors = y_pred_tuned != y_test

if 'importance_df' in locals() and not importance_df.empty:
    top_features = importance_df['feature'].head(5).tolist()
else:
    top_features = X_test.columns[:5].tolist()

rows = []
for feat in top_features:
    rows.append(
        {
            'feature': feat,
            'correct_mean': float(X_test.loc[~errors, feat].mean()),
            'error_mean': float(X_test.loc[errors, feat].mean()),
            'absolute_gap': float(abs(X_test.loc[~errors, feat].mean() - X_test.loc[errors, feat].mean())),
        }
    )

error_feature_df = pd.DataFrame(rows).sort_values('absolute_gap', ascending=False)
display(error_feature_df)


In [ ]:
threshold_rows = []
for threshold in [0.60, 0.70, 0.80, 0.90]:
    metrics = compute_error_analysis(
        model=tuned_model,
        X_test=X_test,
        y_test=y_test,
        confidence_threshold=threshold,
    )
    threshold_rows.append(
        {
            'threshold': threshold,
            'high_confidence_share': metrics.get('high_confidence_share', np.nan),
            'high_confidence_accuracy': metrics.get('high_confidence_accuracy', np.nan),
            'low_confidence_accuracy': metrics.get('low_confidence_accuracy', np.nan),
        }
    )

confidence_strategy_df = pd.DataFrame(threshold_rows)
display(confidence_strategy_df)

save_table_snapshot(
    confidence_strategy_df,
    table_path('08', 'confidence_strategy_thresholds'),
    title='Model Choice - Confidence Threshold Strategy',
    index=False,
)
confidence_strategy_df.to_csv(table_path('08', 'confidence_strategy_thresholds', 'csv'), index=False)

repro_footer = build_reproducibility_footer(cfg.random_state)
display(repro_footer)
save_table_snapshot(
    repro_footer,
    table_path('99', 'reproducibility_footer'),
    title='Model Choice - Reproducibility Footer',
    index=False,
)
repro_footer.to_csv(table_path('99', 'reproducibility_footer', 'csv'), index=False)


## Feature-Based Error Analysis


In [ ]:
# Use top tuned-model features to inspect how errors differ from correct predictions
y_pred_tuned = tuned_model.predict(X_test)

if 'importance_df' in locals() and not importance_df.empty:
    top_features = importance_df['feature'].head(5).tolist()
else:
    top_features = X_test.columns[:5].tolist()

print(f"Top features used for error pattern scan: {top_features[:3]}")
feature_error_patterns(
    X_test=X_test,
    y_test=y_test,
    y_pred=y_pred_tuned,
    top_features=top_features,
)


## Confidence-Based Decision Framework and Calibration


In [ ]:
# Operational strategy: auto-process high-confidence predictions, review low-confidence ones
confidence_strategy(
    model=tuned_model,
    X_test=X_test,
    confidence_threshold=0.80,
)

# Calibration check (binary proxy target required)
if y_test_binary is None:
    print('Binary target not available. Skipping calibration plot.')
else:
    check_calibration(
        model=tuned_model,
        X_test=X_test,
        y_test=y_test_binary,
    )


Create Lurning Curves

## Bootstrap Confidence Intervals

## Expert To-Do

- Add probability calibration report (Brier score + calibration curves).
- Add fairness slice metrics by race/age/payer for tuned model.
- Add decision-threshold optimization linked to hospital intervention capacity.


## SHAP Explanations (3 Example Predictions)

This section gives local explanations for three representative test predictions: a high-confidence correct case, a low-confidence case, and a high-confidence error case (if any errors exist).


In [ ]:
try:
    import shap
except ImportError as e:
    shap = None
    print('SHAP is not installed in this environment. Install with: pip install shap')

if shap is not None:
    # Use the underlying estimator for SHAP; keep feature names from the original DataFrame.
    model_step = tuned_model.named_steps.get('model', tuned_model)
    scaler_step = tuned_model.named_steps.get('scaler') if hasattr(tuned_model, 'named_steps') else None
    feature_names = list(X_train.columns) if hasattr(X_train, 'columns') else [f'f{i}' for i in range(X_train.shape[1])]

    def _transform_for_model(X_df):
        X_arr = X_df.values if hasattr(X_df, "values") else X_df
        if scaler_step is not None:
            return scaler_step.transform(X_arr)
        return X_arr

    # Pick 3 representative examples from the test set.
    if not hasattr(tuned_model, "predict_proba"):
        raise RuntimeError('tuned_model does not support predict_proba; SHAP selection logic expects probabilities.')

    proba = tuned_model.predict_proba(X_test)
    pred = proba.argmax(axis=1)
    conf = proba.max(axis=1)
    y_true = np.asarray(y_test)
    correct = pred == y_true
    mis = ~correct

    def _safe_pick(mask, score, prefer_max=True):
        if not np.any(mask):
            return None
        s = np.where(mask, score, -np.inf if prefer_max else np.inf)
        return int(np.nanargmax(s) if prefer_max else np.nanargmin(s))

    idx_high_conf_correct = _safe_pick(correct, conf, prefer_max=True)
    idx_low_conf = int(np.nanargmin(conf))
    idx_high_conf_error = _safe_pick(mis, conf, prefer_max=True)

    selected = [idx_high_conf_correct, idx_low_conf, idx_high_conf_error]
    selected = [i for i in selected if i is not None]
    # Ensure 3 unique indices (fill with next-best correct cases if needed).
    selected_unique = []
    for i in selected:
        if i not in selected_unique:
            selected_unique.append(i)
    if len(selected_unique) < 3:
        # Add more correct examples by descending confidence.
        fallback = np.argsort(-conf)
        for i in fallback:
            i = int(i)
            if i not in selected_unique:
                selected_unique.append(i)
            if len(selected_unique) == 3:
                break

    class_names = {0: "NO", 1: ">30", 2: "<30"}

    def get_shap_explanation(model, X_sample_df, feature_names, class_idx=None, background_df=None, max_features=10):
        """Print top local SHAP contributions for a single sample.
        Uses TreeExplainer for tree models; falls back to shap.Explainer otherwise.
        """
        X_background_df = background_df if background_df is not None else X_train.sample(200, random_state=cfg.random_state)
        X_bg = _transform_for_model(X_background_df)
        X_s = _transform_for_model(X_sample_df)

        is_tree = model.__class__.__name__ in {"RandomForestClassifier", "XGBClassifier", "LGBMClassifier", "CatBoostClassifier"}
        if is_tree:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_s)
            expected = explainer.expected_value
        else:
            explainer = shap.Explainer(model, X_bg, feature_names=feature_names)
            shap_exp = explainer(X_s)
            shap_values = shap_exp.values
            expected = shap_exp.base_values

        # Extract a 1D vector for the requested class.
        if class_idx is None:
            class_idx = 0

        if isinstance(shap_values, list):
            v = np.asarray(shap_values[class_idx])[0]
            base = expected[class_idx] if isinstance(expected, (list, np.ndarray)) else expected
        else:
            arr = np.asarray(shap_values)
            if arr.ndim == 3:
                v = arr[0, :, class_idx]
                base = np.asarray(expected)[0, class_idx]
            else:
                v = arr[0]
                base = np.asarray(expected)[0] if np.asarray(expected).ndim else expected

        order = np.argsort(np.abs(v))[::-1][:max_features]
        print("Top feature contributions (positive pushes towards selected class output):")
        for j in order:
            name = feature_names[int(j)] if int(j) < len(feature_names) else f"f{int(j)}"
            val = float(v[int(j)])
            direction = "increases" if val > 0 else "decreases"
            print(f"{name}: {direction} by {abs(val):.4f}")

        # Optional: plot a SHAP waterfall for this class output if available.
        try:
            exp = shap.Explanation(values=v, base_values=base, data=X_sample_df.iloc[0].values, feature_names=feature_names)
            shap.plots.waterfall(exp, max_display=max_features)
        except Exception as plot_e:
            print(f"(Waterfall plot skipped: {plot_e})")

    # Run SHAP for the 3 selected examples.
    for k, idx in enumerate(selected_unique, 1):
        X_one = X_test.iloc[[idx]]
        y_true_i = int(y_test.iloc[idx]) if hasattr(y_test, "iloc") else int(y_test[idx])
        pred_i = int(pred[idx])
        conf_i = float(conf[idx])
        status = "correct" if pred_i == y_true_i else "error"
        print("" + "=" * 80)
        print(f"Example {k}: idx={idx} | true={class_names.get(y_true_i, y_true_i)} | pred={class_names.get(pred_i, pred_i)} | conf={conf_i:.3f} | {status}")
        print("Using SHAP class output:", class_names.get(pred_i, pred_i))
        get_shap_explanation(model_step, X_one, feature_names, class_idx=pred_i)
